# DP Simulation of CIGRE MV with PV Inverters and Load Step

In [ ]:
import requests
import glob

def download_grid_data(name, url):
    with open(name, 'wb') as out_file:
        content = requests.get(url, stream=True).content
        out_file.write(content)

url = 'https://raw.githubusercontent.com/dpsim-simulator/cim-grid-data/master/CIGRE_MV/NEPLAN/CIGRE_MV_no_tapchanger_noLoad1_LeftFeeder_With_LoadFlow_Results/Rootnet_FULL_NE_28J17h'
filename = 'CIGRE-MV'
download_grid_data(filename+'_EQ.xml', url+'_EQ.xml')
download_grid_data(filename+'_TP.xml', url+'_TP.xml')
download_grid_data(filename+'_SV.xml', url+'_SV.xml')

files = glob.glob(filename+'_*.xml')
print(files)

In [ ]:
from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
import matplotlib.pyplot as plt
import dpsimpy

In [ ]:
time_step = 1e-3
final_time = 3.0
sim_name = 'DP_CIGRE_MV_withDG_withLoadStep'
sim_name_pf = sim_name + "_Powerflow"

### Powerflow for Initialization

In [ ]:
dpsimpy.Logger.set_log_dir('logs/' + sim_name_pf)
reader = dpsimpy.CIMReader(sim_name_pf, dpsimpy.LogLevel.debug, dpsimpy.LogLevel.debug)
system_pf = reader.loadCIM(50, files, dpsimpy.Domain.SP, dpsimpy.PhaseType.Single, dpsimpy.GeneratorType.PVNode)
pv_active_power = 50e3 * int(int(4319.1e3 / 50e3) / 9)
pv_reactive_power = np.sqrt(np.power(pv_active_power / 1, 2) - np.power(pv_active_power, 2))

for n in range(3, 12):
    connection_node = system_pf.node('N' + str(n))
    pv = dpsimpy.sp.ph1.AvVoltageSourceInverterDQ('pv_' + connection_node.name(), 'pv_' + connection_node.name(), dpsimpy.LogLevel.debug, True)
    pv.set_parameters(sys_omega=2 * np.pi * 50, sys_volt_nom=1500, p_ref=pv_active_power, q_ref=pv_reactive_power)
    pv.set_controller_parameters(Kp_pll=0.25 / 10, Ki_pll=2 / 1000,
                                Kp_power_ctrl=0.001 / 10, Ki_power_ctrl=0.08 / 1000 ,
                                Kp_curr_ctrl=0.3 / 10, Ki_curr_ctrl=10 / 1000, omega_cutoff=2*np.pi*50)
    pv.set_filter_parameters(Lf=0.002,Cf=789.3e-6,Rf=0.1,Rc=0.1)
    pv.set_transformer_parameters(nom_voltage_end_1=20e3, nom_voltage_end_2=1500, rated_power = 5e6,
                                ratio_abs=20e3 / 1500, ratio_phase=0,resistance=0, inductance=0.928e-3)
    pv.set_initial_state_values(p_init=450000.716605, q_init=-0.577218, phi_d_init=3854.197405 * 1000, phi_q_init=-0.003737 * 1000, gamma_d_init=128.892668 * 1000, gamma_q_init=23.068682 * 1000)
    system_pf.add(pv)
    system_pf.connect_component(pv, [connection_node])


logger_pf = dpsimpy.Logger(sim_name_pf)
for node in system_pf.nodes:
    logger_pf.log_attribute(node.name() + '.V', 'v', node)

sim_pf = dpsimpy.Simulation(sim_name_pf, dpsimpy.LogLevel.debug)
sim_pf.set_system(system_pf)
sim_pf.set_time_step(1.0)
sim_pf.set_final_time(2.0)
sim_pf.set_domain(dpsimpy.Domain.SP)
sim_pf.set_solver(dpsimpy.Solver.NRP)
sim_pf.set_solver_component_behaviour(dpsimpy.SolverBehaviour.Initialization)
sim_pf.do_init_from_nodes_and_terminals(True)

sim_pf.add_logger(logger_pf)
sim_pf.run()


### Dynamic simulation

In [ ]:
dpsimpy.Logger.set_log_dir('logs/' + sim_name)
reader2 = dpsimpy.CIMReader(sim_name, dpsimpy.LogLevel.info, dpsimpy.LogLevel.debug)
system_dp = reader2.loadCIM(50, files, dpsimpy.Domain.DP, dpsimpy.PhaseType.Single, dpsimpy.GeneratorType.PVNode)

pv_active_power = 50e3 * int(int(4319.1e3 / 50e3) / 9)
pv_reactive_power = np.sqrt(np.power(pv_active_power / 1, 2) - np.power(pv_active_power, 2))

for n in range(3, 12):
    connection_node = system_dp.node('N' + str(n))
    pv = dpsimpy.dp.ph1.AvVoltageSourceInverterDQ('pv_' + connection_node.name(), 'pv_' + connection_node.name(), dpsimpy.LogLevel.debug, True)
    pv.set_parameters(sys_omega=2 * np.pi * 50, sys_volt_nom=1500, p_ref=pv_active_power, q_ref=pv_reactive_power)
    pv.set_controller_parameters(Kp_pll=0.25 / 10, Ki_pll=2 / 1000,
                                Kp_power_ctrl=0.001 / 10, Ki_power_ctrl=0.08 / 1000 ,
                                Kp_curr_ctrl=0.3 / 10, Ki_curr_ctrl=10 / 1000, omega_cutoff=2*np.pi*50)
    pv.set_filter_parameters(Lf=0.002,Cf=789.3e-6,Rf=0.1,Rc=0.1)
    pv.set_transformer_parameters(nom_voltage_end_1=20e3, nom_voltage_end_2=1500, rated_power = 5e6,
                                ratio_abs=20e3 / 1500, ratio_phase=0,resistance=0, inductance=0.928e-3)
    pv.set_initial_state_values(p_init=450000.716605, q_init=-0.577218, phi_d_init=3854.197405 * 1000, phi_q_init=-0.003737 * 1000, gamma_d_init=128.892668 * 1000, gamma_q_init=23.068682 * 1000)
    system_dp.add(pv)
    system_dp.connect_component(pv, [connection_node])

system_dp.init_with_powerflow(system_pf)

# log node voltages
logger_dp= dpsimpy.Logger(sim_name)
for node in system_dp.nodes:
    logger_dp.log_attribute(node.name() + '.V', 'v', node)

# log line and load currents
for comp in system_dp.components:
    if isinstance(comp, dpsimpy.dp.ph1.PiLine):
        logger_dp.log_attribute(comp.name() + '.I', 'i_intf', comp)
    if isinstance(comp, dpsimpy.dp.ph1.RXLoad):
        logger_dp.log_attribute(comp.name() + '.I', 'i_intf', comp)

# log output of PV connected at N11
pv_name = 'pv_N11'
pv = system_dp.component(pv_name)
input_names = [
    "pv_powerctrl_input_pref", "pv_powerctrl_input_qref", "pv_powerctrl_input_vcd",
    "pv_powerctrl_input_vcq", "pv_powerctrl_input_ircd", "pv_powerctrl_input_ircq"
]
logger_dp.log_attribute(input_names, 'powerctrl_inputs', pv)

state_names = [
    "pv_powerctrl_state_p", "pv_powerctrl_state_q", "pv_powerctrl_state_phid",
    "pv_powerctrl_state_phiq", "pv_powerctrl_state_gammad", "pv_powerctrl_state_gammaq"
]
logger_dp.log_attribute(state_names, 'powerctrl_states', pv)

output_names = [
    "pv_powerctrl_output_vsd", "pv_powerctrl_output_vsq"
]

logger_dp.log_attribute(output_names, 'powerctrl_outputs', pv)

logger_dp.log_attribute(pv_name + '_v_intf', 'v_intf', pv)
logger_dp.log_attribute(pv_name + '_i_intf', 'i_intf', pv)
logger_dp.log_attribute(pv_name + '_pll_output', 'pll_output', pv)
logger_dp.log_attribute(pv_name + '_vsref', 'Vsref', pv)
logger_dp.log_attribute(pv_name + '_vs', 'Vs', pv)

# load step sized in absolute terms
load_switch = dpsimpy.dp.ph1.Switch("Load_Add_Switch_N11", dpsimpy.LogLevel.debug)
connection_node = system_dp.node('N11')
resistance = np.abs(connection_node.initial_single_voltage())**2 / 1500.0e3
load_switch.set_parameters(1e9, resistance)
load_switch.open()
system_dp.add(load_switch)
system_dp.connect_component(load_switch, [dpsimpy.dp.SimNode.gnd, system_dp.node('N11')])
logger_dp.log_attribute('switchedload_i', 'i_intf', load_switch)
load_step_event = dpsimpy.event.SwitchEvent(2  - time_step, load_switch, True)


sim_dp = dpsimpy.Simulation(sim_name, dpsimpy.LogLevel.debug)
sim_dp.set_system(system_dp)
sim_dp.set_time_step(time_step)
sim_dp.set_final_time(final_time)
sim_dp.set_domain(dpsimpy.Domain.DP)
sim_dp.set_solver(dpsimpy.Solver.MNA)
sim_dp.add_event(load_step_event)
sim_dp.do_steady_state_init(False)
sim_dp.do_init_from_nodes_and_terminals(True)

sim_dp.add_logger(logger_dp)
sim_dp.run()

## Powerflow

### Read results

In [ ]:
modelName = 'DP_CIGRE_MV_withDG_withLoadStep_Powerflow'
path = 'logs/' + modelName + '/'
dpsim_result_file = path + modelName + '.csv'

ts_dpsim = read_timeseries_csv(dpsim_result_file)

### Plot results

In [ ]:
plt.figure(figsize=(12,6))
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name != 'N0.V':
        plt.plot(ts_obj.time, ts_obj.abs().values, label=ts_name)
plt.legend()
plt.show()

## Dynamic Phasor

In [ ]:
modelName = 'DP_CIGRE_MV_withDG_withLoadStep'
path = 'logs/' + modelName + '/'
dpsim_result_file = path + modelName + '.csv'

ts_dpsim = read_timeseries_csv(dpsim_result_file)

### Plot results

In [ ]:
plt.figure(figsize=(12,6))
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name != 'N0.V' and ts_name[-2:] == '.V':
        plt.plot(ts_obj.time, ts_obj.abs().values, label=ts_name)
plt.legend()
plt.show()

## Voltages at first timestep

In [ ]:
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name[-2:] == '.V':
        print(ts_name + ': ' + str(ts_obj.abs().values[0]) + ', ' + str(ts_obj.phase().values[0]))

### Plot zoomed to Load Step

In [ ]:
plt.figure(figsize=(12,6))
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name != 'N0.V' and ts_name[-2:] == '.V':
        plt.plot(ts_obj.time, ts_obj.abs().values, label=ts_name)
#plt.ylim([19000,21000])
plt.xlim([1.9,3])
plt.legend()
plt.show()

### DP backshift

In [ ]:
plt.figure(figsize=(12,12))
subplot_number = 1
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name[-2:] == '.V' and ts_name[:-2] != 'N0.V':
        plt.subplot(4,3,subplot_number)
        subplot_number += 1
        plt.plot(ts_dpsim[ts_name].interpolate(50e-6).time, ts_dpsim[ts_name].interpolate(50e-6).frequency_shift(50).values, label=ts_name)
        plt.legend(loc='upper right')
        plt.xlim([1.8,3])
        plt.ylim([18500,20000])
plt.show()

## Load currents

In [ ]:
plt.figure(figsize=(12,6))
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name[:4]=='LOAD':        
        plt.plot(ts_obj.time, ts_obj.abs().values, label=ts_name)
#plt.xlim([0,0.1])
plt.legend(loc='upper right')
plt.show()

## Line currents

In [ ]:
plt.figure(figsize=(12,6))
subplot_number = 1
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name[-2:] == '.I' and ts_name[:4] != 'LOAD':
        plt.subplot(4,3,subplot_number)
        plt.plot(ts_obj.time, ts_obj.abs().values, label=ts_name)
        subplot_number += 1
        plt.legend(loc='upper right')
#plt.xlim([0,0.1])
plt.show()

## PV inverter at N11

### States - Powers

In [ ]:
plt.figure(figsize=(12,6))
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name[-7:]=='state_p' or ts_name[-7:]=='state_q':
        plt.plot(ts_obj.time, ts_obj.abs().values, label=ts_name)
plt.legend()
plt.show()

### States - Others

In [ ]:
plt.figure(figsize=(12,6))
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name[-7:]!='state_p' and ts_name[-7:]!='state_q' and  'state' in ts_name:
        plt.plot(ts_obj.time, ts_obj.abs().values, label=ts_name)
plt.legend()
plt.show()

### States at first timestep

In [ ]:
for ts_name, ts_obj  in ts_dpsim.items():
    if 'state' in ts_name:
        print(ts_name + ': ' + str(ts_obj.abs().values[0]) + ', ' + str(ts_obj.phase().values[0]))

### States at last timestep

In [ ]:
for ts_name, ts_obj  in ts_dpsim.items():
    if 'state' in ts_name:
        print(ts_name + ': ' + str(ts_obj.abs().values[-1]) + ', ' + str(ts_obj.phase().values[-1]))

### Plot inputs

In [ ]:
plt.figure(figsize=(12,6))
for ts_name, ts_obj  in ts_dpsim.items():
    if 'input' in ts_name:
        plt.plot(ts_obj.time, ts_obj.abs().values, label=ts_name)
plt.legend()
plt.show()

### Inputs at first timestep

In [ ]:
for ts_name, ts_obj  in ts_dpsim.items():
    if 'input' in ts_name:
        print(ts_name + ': ' + str(ts_obj.abs().values[0]) + ', ' + str(ts_obj.phase().values[0]))

### Inputs at last timestep

In [ ]:
for ts_name, ts_obj  in ts_dpsim.items():
    if 'input' in ts_name:
        print(ts_name + ': ' + str(ts_obj.abs().values[-1]) + ', ' + str(ts_obj.phase().values[-1]))

### Plot outputs

In [ ]:
plt.figure(figsize=(12,6))
for ts_name, ts_obj  in ts_dpsim.items():
    if 'output' in ts_name:
        plt.plot(ts_obj.time, ts_obj.abs().values, label=ts_name)
plt.legend()
plt.show()

### Outputs at first timestep

In [ ]:
for ts_name, ts_obj  in ts_dpsim.items():
    if 'output' in ts_name:
        print(ts_name + ': ' + str(ts_obj.abs().values[0]) + ', ' + str(ts_obj.phase().values[0]))

### Outputs at last timestep

In [ ]:
for ts_name, ts_obj  in ts_dpsim.items():
    if 'output' in ts_name:
        print(ts_name + ': ' + str(ts_obj.abs().values[-1]) + ', ' + str(ts_obj.phase().values[-1]))

### Plot interface vars

In [ ]:
plt.figure(figsize=(12,6))
for ts_name, ts_obj  in ts_dpsim.items():
    if 'intf' in ts_name:
        plt.plot(ts_obj.time, ts_obj.abs().values, label=ts_name)
plt.legend()
plt.show()

## Plot derived power

In [ ]:
pv_N11_s_intf = ts_dpsim['pv_N11_v_intf'].values*np.conj(ts_dpsim['pv_N11_i_intf'].values)
plt.figure(figsize=(12,6))
plt.plot(ts_dpsim['pv_N11_v_intf'].time, np.real(pv_N11_s_intf), label='pv_N11_P_intf')
plt.plot(ts_dpsim['pv_N11_v_intf'].time, np.imag(pv_N11_s_intf), label='pv_N11_Q_intf')
plt.legend()
plt.show()


### Interface vars at first timestep

In [ ]:
for ts_name, ts_obj  in ts_dpsim.items():
    if 'intf' in ts_name:
        print(ts_name + ': ' + str(ts_obj.abs().values[0]) + ', ' + str(ts_obj.phase().values[0]))
print('pv_N11_s_intf' + ': ' + str(np.real(pv_N11_s_intf[0])) + ', ' + str(np.imag(pv_N11_s_intf[0])))

### Interface vars at last timestep

In [ ]:
for ts_name, ts_obj  in ts_dpsim.items():
    if 'intf' in ts_name:
        print(ts_name + ': ' + str(ts_obj.abs().values[-1]) + ', ' + str(ts_obj.phase().values[-1]))
print('pv_N11_s_intf' + ': ' + str(np.real(pv_N11_s_intf[-1])) + ', ' + str(np.imag(pv_N11_s_intf[-1])))